In [11]:
import torch

In [12]:
def is_leap_year(y: int) -> bool:
    if y % 400 == 0:
        return True
    if y % 100 == 0:
        return False
    return y % 4 == 0

In [ ]:
from collections import OrderedDic
from torch import nn

class LeapNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(OrderedDict([
            ('hidden', nn.Linear(3, 4)),     # Single hidden layer
            ('relu', nn.ReLU()),
            ('output', nn.Linear(4, 1)),
            ('sigmoid', nn.Sigmoid())
        ]))

    def forward(self, x):
        return self.stack(x)

    def encode_year(year: int) -> torch.Tensor:
        # We are ok with years 0-65535 (16-bit)
        assert(0 <= year <= 65535)
        return torch.tensor([
            float(year % 4 == 0),    # divisible by 4
            float(year % 100 == 0),  # divisible by 100
            float(year % 400 == 0)   # divisible by 400
        ], dtype=torch.float32)

In [25]:
def train_and_validate(model_class: nn.Module) -> tuple[nn.Module, torch.Tensor]:
    # Create training data for all possible years
    years = torch.arange(0, 65536)
    X = torch.stack([model_class.encode_year(int(y)) for y in years])
    y = torch.tensor([float(is_leap_year(int(y))) for y in years], dtype=torch.float32).reshape(-1, 1)
    
    # Initialize model and optimizer
    model = model_class()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    criterion = nn.BCELoss()
    
    # Train for 100 epochs
    for epoch in range(100):
        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item():.6f}')
    
    # Get predictions for all inputs
    with torch.no_grad():
        all_predictions = model(X)
    
    return model, all_predictions

In [ ]:
def run_experiment(model_class):
    # Train the model and get predictions
    model, predictions = train_and_validate(model_class)

    # Calculate accuracy
    years = torch.arange(0, 65536)
    true_labels = torch.tensor([float(is_leap_year(int(y))) for y in years], dtype=torch.float32).reshape(-1, 1)
    predicted_labels = (predictions >= 0.5).float()
    accuracy = (predicted_labels == true_labels).float().mean()

    print(f"Overall accuracy: {accuracy.item():.6f}")

    # Show some example predictions
    print("\nSample predictions (year: true -> predicted):")
    for year in [2020, 2021, 2000, 1900, 2100]:
        with torch.no_grad():
            pred = model(model_class.encode_year(year))
        print(f"{year}: {is_leap_year(year)} -> {pred.item():.6f}")


In [18]:
run_experiment(LeapNet)

Epoch 10, Loss: 0.683772
Epoch 20, Loss: 0.606247
Epoch 30, Loss: 0.508997
Epoch 40, Loss: 0.402763
Epoch 50, Loss: 0.303671
Epoch 60, Loss: 0.219485
Epoch 70, Loss: 0.155203
Epoch 80, Loss: 0.110464
Epoch 90, Loss: 0.080003
Epoch 100, Loss: 0.059809
Overall accuracy: 1.000000

Sample predictions (year: true -> predicted):
2020: True -> 0.933937
2021: False -> 0.052544
2000: True -> 0.918699
1900: False -> 0.137901
2100: False -> 0.137901


In [23]:
from collections import OrderedDict
from torch import nn

class LeapNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(OrderedDict([
            ('hidden', nn.Linear(3, 12)),     # Single hidden layer
            ('relu', nn.ReLU()),
            ('output', nn.Linear(12, 1)),
            ('sigmoid', nn.Sigmoid())
        ]))

    def forward(self, x):
        return self.stack(x)

    def encode_year(year: int) -> torch.Tensor:
        # We are ok with years 0-65535 (16-bit)
        assert(0 <= year <= 65535)
        return torch.tensor([year, year, year], dtype=torch.float32)

In [26]:
run_experiment(LeapNet2)

Epoch 10, Loss: 24.223743
Epoch 20, Loss: 24.218676
Epoch 30, Loss: 24.166487
Epoch 40, Loss: 24.208176
Epoch 50, Loss: 24.226738
Epoch 60, Loss: 24.230324
Epoch 70, Loss: 24.231634
Epoch 80, Loss: 24.231955
Epoch 90, Loss: 24.232063
Epoch 100, Loss: 24.232092
Overall accuracy: 0.757523

Sample predictions (year: true -> predicted):
2020: True -> 0.000000
2021: False -> 0.000000
2000: True -> 0.000000
1900: False -> 0.000000
2100: False -> 0.000000
